In [1]:
import pandas as pd
import sqlalchemy

In [2]:
COL_SEPARATOR = "_123456789987654321_"

In [3]:
SQL_CREDENTIALS = "root:1234"
engine = sqlalchemy.create_engine('mysql+pymysql://' + SQL_CREDENTIALS + '@localhost:3306/music_recommender_db')

# get tracks with acoustic features and genres

In [97]:
stmt = "SELECT track_id, danceability, energy, speechiness, acousticness, instrumentalness, tempo, valence, liveness FROM acoustic_features"
tracks_with_af_df = pd.read_sql(sql=stmt, con=engine)
tracks_with_af_df.head()

,track_id,danceability,energy,speechiness,acousticness,instrumentalness,tempo,valence,liveness
0,1,0.463,0.877,0.0307,0.000111,0.000006,173.135,0.698,0.523
1,6,0.457,0.387,0.0314,0.728000,0.127000,120.011,0.041,0.114
2,7,0.268,0.903,0.0644,0.025200,0.016200,154.988,0.464,0.206
3,9,0.502,0.980,0.0591,0.000021,0.025600,121.962,0.388,0.102
4,11,0.200,0.275,0.0478,0.030200,0.606000,84.479,0.248,0.850


In [98]:
tracks_with_af_df.dropna(inplace=True)

In [99]:
tracks_with_afs = tracks_with_af_df["track_id"].unique()
len(tracks_with_afs)

3468289

In [100]:
tracks_with_genres_df = pd.read_csv("feature_engineering/data/artist_track_genres_prepared.csv", sep=COL_SEPARATOR, header=None)
tracks_with_genres_df.columns = ["track_id", "title", "artist_id", "artist", "playcount", "genres"]
tracks_with_genres_df.dropna(inplace=True)
tracks_with_genres_df["genres"] = tracks_with_genres_df["genres"].apply(lambda r: str(r).split("\t"))
tracks_with_genres_df.head()

C:\Users\pmuellner\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,track_id,title,artist_id,artist,playcount,genres
0,11393952,Somebody That I Used to Know,3085,Gotye,185187,"[electronic, pop, indiepop, rock, singersongwr..."
1,4868,Rolling in the Deep,504,Adele,171202,"[soul, pop, singersongwriter, blues, jazz, ind..."
2,2900,Intro,32,The xx,130971,"[electronic, indiepop, shoegaze, dreampop, pos..."
3,2498,Blue Jeans,402,Lana Del Rey,130912,"[pop, soul, indiepop, chamberpop, triphop, dre..."
4,26142,We Are Young (feat. Janelle MonÃ¡e),3151,fun.,119257,"[pop, indiepop, rock, alternativerock, alterna..."


In [101]:
tracks_with_genres = tracks_with_genres_df["track_id"].tolist()

In [102]:
len(tracks_with_genres)

320504

In [103]:
relevant_tracks = set(tracks_with_afs).intersection(tracks_with_genres)

In [104]:
len(relevant_tracks)

167706

In [90]:
relevant_tracks_df = pd.DataFrame(data=relevant_tracks, columns=["track_id"])
relevant_tracks_df.head()

,track_id
0,524289
1,6
2,7
3,524295
4,9


In [56]:
relevant_tracks_df.to_csv("tracks_with_afs_genres.csv", sep=";", index=False)

# get users with country and mainstreaminess

In [57]:
stmt = "SELECT user_id, country, M_global_R_APC FROM user_mainstreaminess"
relevant_users_df = pd.read_sql(con=engine, sql=stmt)
relevant_users_df.dropna(inplace=True)
relevant_users_df.head()

,user_id,country,M_global_R_APC
0,384,UK,0.166818
1,3653,UK,0.192014
2,4813,US,0.096758
3,5069,AT,0.251059
4,5213,UK,0.256085


In [58]:
relevant_users = relevant_users_df["user_id"].tolist()

In [59]:
len(relevant_users)

53252

In [60]:
relevant_users_df.to_csv("users_with_country_main.csv", sep=";", index=False)

# remove dominant genres

In [61]:
dominant_genres = ["rock", "pop", "electronic", "metal", "alternativerock", "indierock"]

In [62]:
cleaned_tracks_df = tracks_with_genres_df[["track_id", "genres"]].copy()
cleaned_tracks_df = cleaned_tracks_df[cleaned_tracks_df["track_id"].isin(relevant_tracks)]
cleaned_tracks_df["genres"] = cleaned_tracks_df["genres"].apply(lambda genres: [g for g in genres if g not in dominant_genres])
dominant_tracks_df = cleaned_tracks_df[cleaned_tracks_df["genres"].apply(len) == 0]
cleaned_tracks_df = cleaned_tracks_df[cleaned_tracks_df["genres"].apply(len) != 0]

In [63]:
len(relevant_tracks)

167706

In [64]:
len(cleaned_tracks_df)

161650

In [65]:
dominant_tracks = dominant_tracks_df["track_id"].values.tolist()

In [66]:
print("%d tracks with no nondominant genre" % len(dominant_tracks))

6056 tracks with no nondominant genre


In [67]:
dominant_tracks_df["track_id"].to_csv("dominant_tracks.csv", sep=";", index=False)

C:\Users\pmuellner\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [68]:
lowms = pd.read_csv("surprise_recommendations/data/low_main_users.txt", usecols=["user_id"], squeeze=True).to_numpy()
lowms

array([ 1049656,  1055118,  1056935, ..., 49951110, 49969247, 50071075],
      dtype=int64)

In [69]:
all_track_ids = cleaned_tracks_df["track_id"].unique()

In [70]:
lowms_tracks_df = pd.read_csv("feature_engineering/data/user_track.csv", sep=";")
lowms_tracks_df.head()

,user_id,track_id,artist_id
0,31435741,53,21
1,31435741,86,32
2,31435741,127,44
3,31435741,182,57
4,31435741,219,65


In [71]:
lowms_tracks_df[lowms_tracks_df["track_id"].isin(all_track_ids)]["track_id"].nunique()

145131

In [72]:
relevant_tracks = set(relevant_tracks_df["track_id"]).difference(dominant_tracks)
len(relevant_tracks)

161650

In [73]:
lowms_tracks_df[lowms_tracks_df["track_id"].isin(relevant_tracks)]["track_id"].nunique()

145131

In [79]:
pd.DataFrame(relevant_tracks, columns=["track_id"]).to_csv("surprise_recommendations/data/relevant_tracks.csv", index=False)

In [47]:
tracks_with_af_df.isna().sum(axis=0)

track_id            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
dtype: int64

In [95]:
lowms_les_df = pd.read_csv("feature_engineering/data/lowms_les.csv", sep=";")
lowms_les_df.head()

,user_id,track_id,artist_id,timestamp
0,31435741,53,21,1370977938
1,31435741,53,21,1370977728
2,31435741,53,21,1370977518
3,31435741,53,21,1370977308
4,31435741,53,21,1370977098


In [96]:
lowms_les_df[lowms_les_df["track_id"].isin(relevant_tracks)]["track_id"].nunique()

157444

In [81]:
user_set = set(lowms)
statement_events = "SELECT user_id, track_id, artist_id, timestamp FROM events WHERE user_id IN " + str(tuple(user_set))
df = pd.read_sql(con=engine, sql=statement_events)
df.head()

,user_id,track_id,artist_id,timestamp
0,31435741,53,21,1370977938
1,31435741,53,21,1370977728
2,31435741,53,21,1370977518
3,31435741,53,21,1370977308
4,31435741,53,21,1370977098


In [91]:
df_ = df[df["track_id"].isin(relevant_tracks)]
df_.head()

,user_id,track_id,artist_id,timestamp
0,31435741,53,21,1370977938
1,31435741,53,21,1370977728
2,31435741,53,21,1370977518
3,31435741,53,21,1370977308
4,31435741,53,21,1370977098


In [92]:
df_["user_id"].nunique(), df_["track_id"].nunique()

(2073, 163105)

In [93]:
df_ = df_[df_["track_id"].isin(cleaned_tracks_df["track_id"])]

In [94]:
df_["user_id"].nunique(), df_["track_id"].nunique()

(2073, 157444)